### PJ1 动手训练一个词向量

- 训练语料：[百度百科语料](https://elearning.fudan.edu.cn/courses/72208/files/4462367/download?download_frd=1)，约1.5G

- `Python` 相关工具

    - `Json` 解析

    - [结巴分词](https://github.com/fxsjy/jieba)

    - [词向量训练](https://radimrehurek.com/gensim/models/word2vec.html)

- 基本要求（必做）

    - 利用提供的语料训练得到一个 `Word2Vec` 词向量模型，并分析其效果

    - 提交一份相关技术报告，需要包含
    
        - 对所写的代码的解释
        
        - 记录实验过程和心得体会
        
        - 对课堂所学知识的理解

        - 对实验结果的分析

- 其他（选做）

    - 鼓励查找相关资料，探讨如何降低训练代价，提升训练性能

    - 鼓励分析 `Word2Vec` 中不同超参数对最后分词效果的影响（如窗口大小、迭代次数、向量维度等）

    - 鼓励自主探索 `Word2Vec` 的两种不同实现方式（即 `CBOW` 和 `skip-gram`），并分析其优劣

    - 鼓励进行数据分析，探讨如何更高效地利用数据

    - 鼓励对结果进行可视化分析

    - 鼓励基于本次实验进一步拓展，自主完成其他感兴趣的实验

    - 以及分享其他在实验过程中发现的有趣现象

#### 下载数据（略）

- 将数据从 `Elearning` 下载到本地

#### 安装相关的依赖包

- `python` 安装教程请直接在线搜索

- `json` 是 `python` 的内置库，无需额外安装

- 本实验需要用到 `jieba` 和 `gensim`，需要提前安装

In [3]:
%pip install jieba gensim

Note: you may need to restart the kernel to use updated packages.


#### 数据处理

- 使用 `json` 读取数据

In [ ]:
import json

data = []

with open('baike_qa_train.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        if line != '':
            sample = json.loads(line)
            data.append(sample.copy())

- 打印数据，查看格式

In [12]:
def display(data:list):
    print('len:', len(data))
    for i in range(5):
        print(data[i])

In [13]:
display(data)

len: 1425170
{'qid': 'qid_5982723620932473219', 'category': '教育/科学-理工学科-地球科学', 'title': '人站在地球上为什么没有头朝下的感觉 ', 'desc': '', 'answer': '地球上重力作用一直是指向球心的，因此\r\n只要头远离球心，人们就回感到头朝上。'}
{'qid': 'qid_5679706523376347837', 'category': '娱乐-宠物', 'title': '我的小baby', 'desc': '我的小baby-辛巴。温顺可爱的，两个月大的小家伙，第一次养狗，该注意什么呢？求指教～[爱你]', 'answer': '勤洗澡，养成好的卫生习惯'}
{'qid': 'qid_6610724023825624555', 'category': '娱乐-度假旅游', 'title': '请问这起交通事故是谁的责任居多?小车和摩托车发生事故，在无红绿灯 ', 'desc': '小车和摩托车发生事故，在无红绿灯的十字路口，小停车看看左右，在觉得安全的情况下刹车慢慢以时速10公里左右的速度靠右行驶过路口，好没有出到十字路口正中时，被左边突然快速行驶过来的摩托车撞在车头前，摩托车主摔到膝盖和檫伤脸部，请问这起交通事故是谁的责任居多。如果双方都有责任的话，大概各占几成？~\r', 'answer': '通过没有信号控制的十字路口，应该减速慢性，让右边的车先行，按你说的，摩托车好像在汽车的左边，所以严格来说可能摩托车全责。当然还要看汽车是否证照齐全，是否饮酒等。具体由交警调查后认定。'}
{'qid': 'qid_8948366474478096617', 'category': '电脑/网络-互联网', 'title': '松本面板可以配什么品牌的超五类模块?? ', 'desc': '', 'answer': 'AMP的试试吧，还有普天的。'}
{'qid': 'qid_3057155567155332897', 'category': '生活-美食/烹饪', 'title': '请教怎么能很快能洗干净猪肠?有什么方法 ', 'desc': '有什么方法', 'answer': '先用清水冲一下,在用食盐反复的搓揉,直到肠的黏液全去掉为止,在用清水反复冲洗干净,

- 选取合适的部分作为最终的训练数据（可以探讨）

In [14]:
train_data = [i['answer'] for i in data[:1000]]

display(train_data)

len: 1000
地球上重力作用一直是指向球心的，因此
只要头远离球心，人们就回感到头朝上。
勤洗澡，养成好的卫生习惯
通过没有信号控制的十字路口，应该减速慢性，让右边的车先行，按你说的，摩托车好像在汽车的左边，所以严格来说可能摩托车全责。当然还要看汽车是否证照齐全，是否饮酒等。具体由交警调查后认定。
AMP的试试吧，还有普天的。
先用清水冲一下,在用食盐反复的搓揉,直到肠的黏液全去掉为止,在用清水反复冲洗干净,最好把肠上的油去掉一些再下锅。


- 结巴分词（可以探讨不同的分词方式）

In [15]:
import jieba

seg_data = []
for i in train_data:
    seg_i = jieba.lcut(i)
    seg_data.append(seg_i)

display(seg_data)

len: 1000
['地球', '上', '重力', '作用', '一直', '是', '指向', '球心', '的', '，', '因此', '\r\n', '只要', '头', '远离', '球心', '，', '人们', '就', '回', '感到', '头朝', '上', '。']
['勤', '洗澡', '，', '养成', '好', '的', '卫生习惯']
['通过', '没有', '信号', '控制', '的', '十字路口', '，', '应该', '减速', '慢性', '，', '让', '右边', '的', '车', '先行', '，', '按', '你', '说', '的', '，', '摩托车', '好像', '在', '汽车', '的', '左边', '，', '所以', '严格来说', '可能', '摩托车', '全责', '。', '当然', '还要', '看', '汽车', '是否', '证照', '齐全', '，', '是否', '饮酒', '等', '。', '具体', '由', '交警', '调查', '后', '认定', '。']
['AMP', '的', '试试', '吧', '，', '还有', '普天', '的', '。']
['先', '用', '清水', '冲', '一下', ',', '在', '用', '食盐', '反复', '的', '搓揉', ',', '直到', '肠', '的', '黏液', '全', '去掉', '为止', ',', '在', '用', '清水', '反复', '冲洗', '干净', ',', '最好', '把', '肠', '上', '的', '油', '去掉', '一些', '再', '下锅', '。']


#### 训练

- 直接使用相关包进行训练（可以探讨不同训练方式）

In [16]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=seg_data, vector_size=100, window=5, min_count=1, workers=8)
model.save('word2vec.model')

#### 验证

- 使用课堂讲授的相关知识验证所得模型的优劣，并进一步调整

In [17]:
model.wv.key_to_index # 获得所有词汇组

{'，': 0,
 '的': 1,
 ' ': 2,
 '。': 3,
 '\r': 4,
 '、': 5,
 ',': 6,
 '\u3000': 7,
 '是': 8,
 '\r\n': 9,
 '在': 10,
 '了': 11,
 '你': 12,
 '：': 13,
 '有': 14,
 '.': 15,
 '和': 16,
 '可以': 17,
 '就': 18,
 '也': 19,
 '我': 20,
 '不': 21,
 '）': 22,
 '（': 23,
 ')': 24,
 '都': 25,
 '(': 26,
 '！': 27,
 '会': 28,
 '“': 29,
 '”': 30,
 '2': 31,
 '1': 32,
 '如果': 33,
 '要': 34,
 '一个': 35,
 '上': 36,
 '-': 37,
 '3': 38,
 '为': 39,
 '对': 40,
 '中': 41,
 '好': 42,
 '=': 43,
 '人': 44,
 '等': 45,
 '到': 46,
 '；': 47,
 '很': 48,
 '用': 49,
 '或': 50,
 '+': 51,
 '他': 52,
 '后': 53,
 '没有': 54,
 '自己': 55,
 '？': 56,
 '就是': 57,
 '把': 58,
 '与': 59,
 '而': 60,
 '~': 61,
 '将': 62,
 '多': 63,
 '说': 64,
 '这': 65,
 '/': 66,
 '去': 67,
 '时': 68,
 '可': 69,
 '"': 70,
 '能': 71,
 '个': 72,
 '所以': 73,
 '5': 74,
 '但': 75,
 '还': 76,
 '月': 77,
 '4': 78,
 '做': 79,
 '可能': 80,
 '治疗': 81,
 '日': 82,
 '问题': 83,
 '这个': 84,
 '给': 85,
 '10': 86,
 '年': 87,
 '需要': 88,
 '吃': 89,
 '再': 90,
 '什么': 91,
 '系统': 92,
 '以': 93,
 '下': 94,
 '我们': 95,
 ':': 96,
 '因为': 97,
 '时候

In [23]:
model.wv.vectors.shape # 查看词向量大小

(21558, 100)

In [18]:
model.wv['的'] # 得到词的词向量

array([-1.5698346 ,  1.1034702 ,  0.1396582 ,  1.4998164 , -1.1118265 ,
       -2.8297575 ,  0.5615042 ,  2.2404068 , -1.4903784 , -1.802028  ,
       -0.18213691, -2.4421303 ,  0.07965308,  0.7723    ,  1.169817  ,
       -1.20225   ,  0.85013604, -1.3623807 ,  0.0092062 , -2.530962  ,
        1.776783  ,  0.74232477,  1.350839  , -1.398957  , -0.3458412 ,
        0.24692692, -1.5574671 , -0.09226742, -0.48521617, -0.7519789 ,
        1.7162942 , -0.5305073 ,  0.40934592, -1.6406194 ,  0.11653597,
        1.605797  ,  1.5937049 , -0.7445113 , -1.423545  , -2.6873953 ,
       -0.12889306, -0.72389275, -0.8923072 ,  0.09756673,  0.57310253,
       -0.364879  , -1.4789329 ,  0.22828555,  0.2769081 ,  1.8968172 ,
        1.02741   , -1.7636787 , -1.903095  , -0.4465012 ,  0.09644181,
        0.6801612 ,  0.8308344 ,  0.34183726, -1.0312793 ,  0.85901403,
       -0.7056815 ,  0.72761226,  0.6363439 , -0.3039125 , -1.0506647 ,
        1.5530902 , -0.02418057,  0.69320595, -1.3190293 ,  1.34

In [19]:
model.wv.similarity('中国', '美国') # 计算两个词的余弦相似度

0.9992347

In [20]:
model.wv.most_similar('中国') # 取出与“中国”最相似的10个词

[('系统', 0.9997972846031189),
 ('将', 0.999756395816803),
 ('以', 0.9997316002845764),
 ('即可', 0.9996917247772217),
 ('技术', 0.9996709227561951),
 ('按', 0.9995360374450684),
 ('武器', 0.9995312690734863),
 ('和', 0.9995248913764954),
 ('与', 0.9994613528251648),
 ('设置', 0.9994303584098816)]

In [22]:
model.wv.most_similar(positive=["上海", "中国"], negative=["纽约"]) # 获得 上海-中国+纽约 的词

[('n', 0.9694655537605286),
 ('II', 0.9685978293418884),
 ('避开', 0.9678846597671509),
 ('分析', 0.9675965309143066),
 ('VS', 0.9675793051719666),
 ('vs', 0.9675726294517517),
 ('·', 0.9673287868499756),
 ('天堂', 0.9664450883865356),
 ('风', 0.9660134315490723),
 ('登陆', 0.9658693671226501)]

In [24]:
model.wv.closer_than("上海", "中国") # 比 中国 更接近 上海 的词汇

['.',
 '（',
 '为',
 '等',
 '；',
 '或',
 '"',
 '个',
 '年',
 ':',
 '点',
 '?',
 '．',
 '进行',
 '\n',
 '》',
 '《',
 '克',
 '功能',
 '一',
 '攻击',
 '服务器',
 '…',
 '属性',
 '·',
 '15',
 '以上',
 'C',
 '20',
 '水',
 '米',
 '元',
 '—',
 '内容',
 '30',
 '技术',
 '次',
 '另外',
 '分钟',
 '写字楼',
 '技能',
 'A',
 'NET',
 '干燥',
 '即可',
 ';',
 '炒',
 '魔剑',
 '扬州',
 '超过',
 '美国',
 '武器',
 '比赛',
 '监控',
 '英语',
 '--',
 '净化',
 '基本',
 '网络',
 '表示',
 '为了',
 '北京',
 '２',
 '式',
 '小时',
 '以下',
 '－',
 '天',
 '之间',
 '０',
 '风',
 '等等',
 '各',
 '删除',
 '组',
 '决定',
 '生肖',
 '１',
 '【',
 '不管',
 '均',
 '模式',
 '空竹',
 '之一',
 '每日',
 'VS',
 '３',
 '天堂',
 '就行了',
 '食品',
 '站',
 '量',
 '99',
 '第一',
 '值',
 '50',
 '100',
 '球队',
 '妇女',
 'II',
 '有关',
 '|',
 '％',
 '好评',
 '煮',
 '80',
 '...',
 '成虫',
 'D',
 '路',
 '级',
 '类型',
 '分别',
 '度',
 '洋葱',
 '多种',
 '主战',
 '积分',
 '干',
 '24',
 '相关',
 '60',
 '方向',
 '避开',
 '排名',
 '狗',
 '岁',
 '×',
 '汁',
 '称号',
 '200',
 '暂时',
 '苹果',
 '避免',
 '疗法',
 '牛奶',
 '商务',
 '起',
 'n',
 '线',
 '日本',
 '本',
 '分析',
 '毫米',
 '＋',
 '40',
 '眼睛',
 '商业',
 '亲朋好友',
 '登陆',
 

#### 调整和其他（略）